In [1]:
import time
import asyncio
import concurrent.futures
import requests
import matplotlib.pyplot as plt

RECEIPT_NUMBER_BASE = "EAC2090"
RANGE_START = 108700
RANGE_END = 114300
CASE_TYPE = 'I-765'
CONCURRENCY_LEVEL = 11


def check_status(receipt_num):
    #print(receipt_num)
    pload = {"appReceiptNum": receipt_num, "initCaseSearch":"CHECK STATUS"}
    r = requests.post('https://egov.uscis.gov/casestatus/mycasestatus.do', data=pload)
    if (r.status_code != 200):
        print("Error with {0}, code {1}".format(receipt_num, r.status_code))
        return None
    
    response = r.text.split("\n")
    
    if CASE_TYPE in response[733]:
        return (receipt_num, response[719].strip())
    else:
        return None


def save_plot(status_list):
    x, y = zip(*status_list)
    plt.scatter(x, y)
    plt.show()
    plt.savefig('UscisStats.png', bbox_inches='tight')

status_list = []

async def main():
    num_digits = len(str(RANGE_END))

    loop = asyncio.get_event_loop()
    batches = int((RANGE_END - RANGE_START) / CONCURRENCY_LEVEL + 1)
    print("Running total {} batches".format(batches))
    start = time.time()
    for i in range(batches):
        if i % 10 == 0:
            seconds = time.time() - start
            print(" - processing batch No.{0}, the job has took {1:.2f} seconds.".format(i, seconds))
            print(" - Has found I-765: {}".format(len(status_list)))
        batch_start = RANGE_START + CONCURRENCY_LEVEL * i
        with concurrent.futures.ThreadPoolExecutor(max_workers=CONCURRENCY_LEVEL) as executor:
            loop = asyncio.get_event_loop()
            futures = [
                loop.run_in_executor(
                    executor,
                    check_status,
                    RECEIPT_NUMBER_BASE + str(batch_start+x).zfill(num_digits)
                )
                for x in range(CONCURRENCY_LEVEL)
            ]
            for t in await asyncio.gather(*futures):
                if t is not None:
                    status_list.append(t)
            
                    
    print('{0} takes {1:.2f}% in the range.'.format(CASE_TYPE, 100*len(status_list)/(batches*CONCURRENCY_LEVEL)))

    save_plot(status_list)


loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-1' coro=<main() running at <ipython-input-1-a9979d4de2c1>:40>>

Running total 510 batches
 - processing batch No.0, the job has took 0.00 seconds.
 - Has found I-765: 0
EAC2090108703 <div class="rows text-center">
EAC2090108703 <p>On August 15, 2020, we accepted the fingerprint fee for your Form I-539, Application To Extend/Change Nonimmigrant Status, Receipt Number EAC2090108703. Our Vermont Service Center location is working on your case. We mailed you a notice describing how we will process your case. Please follow the instructions in the notice. If you move, go to <a href="https://egov.uscis.gov/coa/displayCOAForm.do" target="_blank">www.uscis.gov/addresschange</a> to give us your new mailing address.</p>
EAC2090108702 <div class="rows text-center">
EAC2090108702 <p>On August 15, 2020, we accepted the fingerprint fee for your Form I-539, Application To Extend/Change Nonimmigrant Status, Receipt Number EAC2090108702. Our Vermont Service Center location is working on your case. We mailed you a notice describing how we will process your case. Plea

In [4]:
len(status_list)

1480

In [8]:
import json

with open("today.json", 'w') as outfile:
    json.dump(status_list, outfile)